In [ ]:
import numpy as np
import matplotlib.pyplot as plt



In [ ]:
# Consts
# Solar energy flux at Earth
We = 1361 # W/m^2

# Normalization parameters 
# mu_s = 1, D = 1 AU, T such that 

# 2D (r, theta) problem, so 4D state vector and 4D costate

In [ ]:
def f_dyn(t, X):


    # Split into state and costate
    S, L = X[:4], X[4:]

    # Unpack state
    r, theta, vr, vtheta = S
    # Unpack costate
    p_r, p_theta, p_vr, p_vtheta = L

    # Compute optimal control (thrust angle)
    # u = alpha = atan(-3*p_vr + sqrt(9*p_vr**2 + 8*p_vtheta**2)/(4*p_vtheta)
    # T = 2*We/(c*sigma)(Rf^2/r^2)*cos(alpha)
    # beta = 2*We/c*sigma*(Re^2)  ---> T_om = beta/r^2 * cos^2(alpha)
    beta = 2*We/c*sigma*(Re**2)
    alpha = atan(-3*p_vr + np.sqrt(9*p_vr**2 + 8*p_vtheta**2)/(4*p_vtheta))
    # Dynamics
    dS = np.zeros(4)
    dL = np.zeros(4)

    dS[0] = vr
    dS[1] = vtheta/r
    dS[2] = vtheta**2/r - 1/r**2 + beta/r**2 * np.cos(alpha)**3
    dS[3] = -vr*vtheta/r + beta/r**2 * np.cos(alpha)**2 * np.sin(alpha)

    dL[0] = p_theta*vtheta/r**2 + p_vr*(vtheta**2/r**2 - 2/r**3 + 2*beta/r**3*np.cos(alpha)**3) - p_vtheta*(vr*vtheta/r**2 - 2*beta/r**3*np.cos(alpha)**2*np.sin(alpha))
    dL[1] = 0
    dL[2] = -p_r - p_vtheta*vtheta/r
    dL[3] = -p_theta/r - 2*p_vr*vtheta/r + p_vtheta*vr/r
    dL[4] = 0

    return np.concatenate((dS, dL))
